<a href="https://colab.research.google.com/github/yeshshaar/ArticleAnalyzer/blob/main/Fake_News_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df_fake=pd.read_csv('Fake.csv')
df_true=pd.read_csv('True.csv')

In [3]:
df_fake['label']='Fake'
df_true['label']='True'

In [4]:
df=pd.concat([df_fake,df_true],axis=0).sample(frac=1,random_state=42).reset_index(drop=True)

In [5]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",Fake
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",True
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",True
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",Fake
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",True


In [6]:
df['content']=df['title']+" "+df['text']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

In [11]:
x=df['content']
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [12]:
vectorizer=TfidfVectorizer(stop_words=None,max_df=0.8)
x_train_tfidf=vectorizer.fit_transform(x_train)
x_test_tfidf=vectorizer.transform(x_test)

In [13]:
print('Tf idf shape:',x_train_tfidf.shape)

Tf idf shape: (35918, 111987)


In [14]:
nb_model=MultinomialNB()
nb_model.fit(x_train_tfidf,y_train)

MultinomialNB()

In [15]:
y_pred_nb=nb_model.predict(x_test_tfidf)

In [16]:
print('Naive-Bayes Accuracy:',accuracy_score(y_test,y_pred_nb))
print(classification_report(y_test,y_pred_nb))

Naive-Bayes Accuracy: 0.9374164810690423
              precision    recall  f1-score   support

        Fake       0.93      0.95      0.94      4710
        True       0.94      0.92      0.93      4270

    accuracy                           0.94      8980
   macro avg       0.94      0.94      0.94      8980
weighted avg       0.94      0.94      0.94      8980



In [18]:
from sklearn.linear_model import LogisticRegression


lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train_tfidf, y_train)


y_pred_lr = lr_model.predict(x_test_tfidf)


print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.9861915367483296
              precision    recall  f1-score   support

        Fake       0.99      0.98      0.99      4710
        True       0.98      0.99      0.99      4270

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [19]:
def predict_news(text):
  text_tfidf=vectorizer.transform([text])
  prediction=lr_model.predict(text_tfidf)
  return prediction[0]

In [27]:
sample_text=("India Successfully Conducts Test of Agni‑V Missile. On January 10, 2024, India successfully test‑fired its Agni‑V intercontinental ballistic missile from Abdul Kalam Island, off the coast of Odisha. The missile, developed by the Defence Research and Development Organisation (DRDO), has a range of over 5,000 kilometers. Defence officials stated that the test was part of routine user trials and met all mission objectives. Prime Minister Narendra Modi congratulated scientists on the achievement.")
print(predict_news(sample_text))

True
